In [2]:
#%time
import numpy as np
import numpy
import torch       
import torch.nn as nn  
from PIL import Image 
import torchvision.transforms as transforms   # for transforming images into tensors 
import cv2
from PIL import Image as im
from ultralytics import YOLO
#yolo initialization
PATH_load_Yolo="/home/ahmed/CondaWorkSpace/Ahmeddataset/YOLOV8_CURT/best.pt"
model = YOLO(PATH_load_Yolo)  # load an custom model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device: ',device)

#KeyPoint model initialization

PATH_load_keypoints="/home/ahmed/CondaWorkSpace/Ahmeddataset/dataset_k/models/RESNE_input30x30.1.pt"

# convolution block with BatchNormalization
def ConvBlock(in_channels, out_channels):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
             nn.BatchNorm2d(out_channels),
             nn.ReLU(inplace=True)
             #MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
             ]
    return nn.Sequential(*layers)
class ResNet_insize30(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.conv1 = ConvBlock(in_channels, 64)
        self.res1 = nn.Sequential(ConvBlock(64, 64), ConvBlock(64, 64))
        self.res2 = nn.Sequential(ConvBlock(64, 128), ConvBlock(128, 128))
        self.res3 = nn.Sequential(ConvBlock(128, 256), ConvBlock(256, 256))
        self.res4 = nn.Sequential(ConvBlock(256, 512), ConvBlock(512, 512))
        self.conv2=ConvBlock(64, 128)
        self.conv3=ConvBlock(128, 256)
        self.conv4=ConvBlock(256, 512)
        self.classifier = nn.Sequential(
                                       nn.Flatten(),
                                       nn.Linear(512*30*30, num_classes))
        
    def forward(self, xb): # xb is the loaded batch
        out = self.conv1(xb)
        out = self.res1(out) + out
        out = self.res2(out) + self.conv2(out)       
        out = self.res3(out) + self.conv3(out)  
        out = self.res4(out) + self.conv4(out)  
        out = self.classifier(out)
        return out 
model_key = ResNet_insize30(3, 14).to(device=device)
model_key.load_state_dict(torch.load(PATH_load_keypoints))
print("prescription intializatin.. Done")

device:  cuda
prescription intializatin.. Done


In [22]:
img=cv2.imread("YOLOV8_CURT/4.jpg")

In [23]:
#%time

# loud cones into a singel batch
results = model(img)[0]
cones_images= [img[int(box[1]):int(box[3]), int(box[0]):int(box[2])] for box in results.boxes.xyxy]
cones_labels= results.boxes.cls.tolist()
img_batch=[]
cone_img_shapes=[]
def HWC2CHW(img):
    data = im.fromarray(img)
    to_tensor = transforms.ToTensor()
    tensor = to_tensor(data)
    return tensor.numpy()
counter=0
for cone_img in cones_images:
    cone_img_shapes.append(cone_img.shape)
    cone_img = cv2.resize(cone_img, (30, 30)) 
    img_batch.append(HWC2CHW(cone_img))
    counter=counter+1
img_batch=numpy.array(img_batch)
img_batch=torch.from_numpy(img_batch)

# key points  detection mode
key_points = model_key(img_batch.to(device=device))
# reconstract the real size images
def key_point_size_transform(p,image_out_input_X,image_out_input_Y,image_size_out_X, image_size_out_Y):
    counter=0
    for p_elemrnt in p:
        if(counter%2):
            p[counter]=round(p_elemrnt*(image_size_out_Y/image_out_input_Y))
        else:
            p[counter]=round(p_elemrnt*(image_size_out_X/image_out_input_X))
        counter=counter+1
    return p

key_points_list=[]
counter=0
for key_point in key_points:
    key_points_list.append(key_point_size_transform(key_point.tolist(),
                                                    30 ,30 ,cone_img_shapes[counter][1],
                                                    cone_img_shapes[counter][0]))
    counter=counter+1

    
# add  offset to refere the cones images to the input image    
counter=0
for key_point in key_points_list:
    #offset
    X_offset=results.boxes.xyxy[counter][0]
    Y_offset=results.boxes.xyxy[counter][1]
    #
    key_points_list[counter][0]=int(key_point[0]+X_offset)
    key_points_list[counter][1]=int(key_point[1]+Y_offset)
    key_points_list[counter][2]=int(key_point[2]+X_offset)
    key_points_list[counter][3]=int(key_point[3]+Y_offset)
    key_points_list[counter][4]=int(key_point[4]+X_offset)
    key_points_list[counter][5]=int(key_point[5]+Y_offset)
    key_points_list[counter][6]=int(key_point[6]+X_offset)
    key_points_list[counter][7]=int(key_point[7]+Y_offset)
    key_points_list[counter][8]=int(key_point[8]+X_offset)
    key_points_list[counter][9]=int(key_point[9]+Y_offset)
    key_points_list[counter][10]=int(key_point[10]+X_offset)
    key_points_list[counter][11]=int(key_point[11]+Y_offset)
    key_points_list[counter][12]=int(key_point[12]+X_offset)
    key_points_list[counter][13]=int(key_point[13]+Y_offset)
    counter=counter+1
    
    
#important notes:
# key_points_list contains now list of lists where every list are 14 key point for a singel cone
# cones_labels contaions list of cones labels
# key points is [top,mid_L_top ,mid_R_top ,mid_L_bot,mid_R_bot , bot_L, bot_R ]



print("key_points_list:",key_points_list)
print("cones_labels:",cones_labels)


0: 480x640 2 Yellows, 2 Blues, 6.9ms
Speed: 12.5ms preprocess, 6.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)


key_points_list: [[1423, 794, 1411, 823, 1430, 820, 1407, 845, 1431, 839, 1402, 862, 1439, 862], [512, 785, 504, 807, 522, 805, 503, 830, 526, 825, 500, 849, 533, 843], [1206, 675, 1200, 686, 1210, 685, 1200, 694, 1212, 692, 1196, 706, 1214, 705], [819, 675, 815, 685, 823, 684, 814, 695, 825, 693, 813, 704, 828, 702]]
cones_labels: [0.0, 1.0, 0.0, 1.0]


In [24]:
def transform_key_points_list_2_list_of_tubels(key_points_list):
    counter=0
    x=0
    y=0
    tupel=(x,y)
    list_out=[]
    for i in range(0,len(key_points_list),2):
        x=key_points_list[i]
        y=key_points_list[i+1]
        tupel=(x,y)
        list_out.append(tupel)
    return list_out
key_point_list_of_tubels=[]
for key_points_list_singel_list in key_points_list:
    key_point_list_of_tubels.append(transform_key_points_list_2_list_of_tubels(key_points_list_singel_list))
print(key_point_list_of_tubels)

[[(1423, 794), (1411, 823), (1430, 820), (1407, 845), (1431, 839), (1402, 862), (1439, 862)], [(512, 785), (504, 807), (522, 805), (503, 830), (526, 825), (500, 849), (533, 843)], [(1206, 675), (1200, 686), (1210, 685), (1200, 694), (1212, 692), (1196, 706), (1214, 705)], [(819, 675), (815, 685), (823, 684), (814, 695), (825, 693), (813, 704), (828, 702)]]


In [25]:
points_3D = np.array([
 
                      (0.0, 0.0, 0.0),       #top
 
                      (-6.0, -107.0,0.0),  #mid_L_top
 
                      (6.0, -107.0, 0.0),#mid_R_top
 
                      (-58.0, -214.0, 0.0), #mid_L_bot
 
                      (58.0, -214.0, 0.0),#mid_R_bot
 
                      (-110.0, -321.0, 0.0) #bot_L 
 
                      ,(110.0, -321.0, 0.0) #bot_R 
 
                     ])

fx=1400
fy=1400
cx=1104
cy=621
k1=0
k2=0
p1=0
p2=0
k3=0

distortion_coeffs = np.zeros((4,1))
#focal_length = size[1]
#center = (size[1]/2, size[0]/2)
matrix_camera = np.array(
                         [[fx, 0, cx],
                         [0, fy, cy],
                         [0, 0, 1] ], dtype = "double"
                         )

distortion_coeffs[0]=k1
distortion_coeffs[1]=k2
prescription_output=[]
counter=0
for singel_cone_tubels in key_point_list_of_tubels:
    
    image_points_2D= np.array(singel_cone_tubels, dtype="double")
    success, vector_rotation, vector_translation = cv2.solvePnP(points_3D, image_points_2D, matrix_camera, distortion_coeffs, flags=0)
    if success:
           prescription_output.append((cones_labels[counter],vector_translation))
    counter=counter+1

prescription_output

[(0.0,
  array([[     1539.1],
         [     855.44],
         [     6784.1]])),
 (1.0,
  array([[    -3117.9],
         [     867.23],
         [     7368.4]])),
 (0.0,
  array([[     1065.9],
         [     564.65],
         [      14673]])),
 (1.0,
  array([[    -3247.9],
         [     616.46],
         [      15933]]))]